# to get start
- source .venv/bin/activate or use conda env
- jupyter notebook --port 8899 --ip 0.0.0.0 [in new terminal, not in vscode]

[recommand]
- also, we can you conda activate openpi. then select env upper right.

# pi0 deployment details

### domain shitf
- wrist camera mount
- camera type: ZED
- resilution: In droid: (180, 320, 3) | realsense (480, 640,3)

In [2]:
import dataclasses

import jax

from openpi.models import model as _model
from openpi.policies import droid_policy
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

import Pyro5.api
import numpy as np

# Policy inference

The following example shows how to create a policy from a checkpoint and run inference on a dummy example.

In [3]:
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_fast_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)

# Delete the policy to free up memory.
del policy

print("Actions shape:", result["actions"].shape)

Some kwargs in processor config are unused and will not have any effect: min_token, time_horizon, action_dim, vocab_size, scale. 
Some kwargs in processor config are unused and will not have any effect: min_token, time_horizon, action_dim, vocab_size, scale. 


Actions shape: (10, 8)


In [4]:
config = _config.get_config("pi0_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)

# Delete the policy to free up memory.
del policy

print("Actions shape:", result["actions"].shape)

Actions shape: (10, 8)


## RPC communication

load model

In [5]:
config = _config.get_config("pi0_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)



In [ ]:
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_fast_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

In [ ]:
# delete current policy to free up memory, then load a new one
del policy

In [6]:
# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)
print("Actions shape:", result["actions"].shape)

Actions shape: (10, 8)


In [18]:
import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt



class RealSenseCamera:
    def __init__(self, serial_number, width=640, height=480, fps=30):
        self.serial = serial_number
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_device(self.serial)
        self.config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
        self.pipeline.start(self.config)




    def get_image(self):
        frames = self.pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            return None

        bgr = np.asanyarray(color_frame.get_data())
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)



        return rgb

    def release(self):
        self.pipeline.stop()




# camera serial numbers
serial_left = "948522071060"
serial_wrist = "815412071252"
serial_right = "838212074411"

# # initialize cameras
cam_left = RealSenseCamera(serial_left)
cam_wrist = RealSenseCamera(serial_wrist)
cam_right = RealSenseCamera(serial_right)


In [ ]:

# img_left = cam_left.get_image()
# img_wrist = cam_wrist.get_image()
from openpi_client import image_tools

img_left = image_tools.resize_with_pad(
                        cam_left.get_image(), 224, 224
                    )
img_wrist = image_tools.resize_with_pad(
                        cam_wrist.get_image(), 224, 224
                    )

img_right = image_tools.resize_with_pad(
                        cam_right.get_image(), 224, 224
                    )

# combine the two images
combined = np.hstack((img_left, img_wrist, img_right))  # shape (224, 672, 3)

# display the combined image
plt.figure(figsize=(8, 4))
plt.imshow(combined)
plt.title("Left + Wrist + Right (RGB 224x224)")
plt.axis('off')
plt.show()





In [ ]:
cam_left.release()
cam_wrist.release()
cam_right.release()

In [ ]:
# vla_policy_client
import Pyro5.api
import numpy as np
import time
from openpi_client import image_tools

ns = Pyro5.api.locate_ns()  # Locate the name server
uri = ns.lookup("pi0_controller")  # Use the name server to look up the URI
# uri = "PYRO:obj_556d727dadc34b1f8ca063985999cc83@localhost:36929"  # <-- change URI
controller = Pyro5.api.Proxy(uri)

# prompt
prompt = "pick up the orange toy"

MAX_STEPS = 5000  # maximum number of steps to run
video_buffer = []  # buffer to store video frames

# dummy action
action = np.zeros((10,8), dtype=np.float32)
action_list = action.tolist()  # convert to list for sending

for step in range(MAX_STEPS):
    start = time.time()
    print(f"\n=== Step {step} ===")
    data_to_send = {
        "action": action_list,
        "step": step
    }
    start1 = time.time()
    obs = controller.step(data_to_send)
    end1 = time.time()
    print(f"Controller step took {end1 - start1:.2f} seconds")

    img_left = image_tools.resize_with_pad(
                            cam_left.get_image(), 224, 224
                        )
    img_wrist = image_tools.resize_with_pad(
                            cam_wrist.get_image(), 224, 224
                        )
    img_right = image_tools.resize_with_pad(
                            cam_right.get_image(), 224, 224
                        )
    
    # save the images
    combined = np.hstack([img_left, img_right, img_wrist])
    video_buffer.append(combined)

    observation = {
        # "observation/exterior_image_1_left": np.array(obs["image_left"], dtype=np.uint8),
        # "observation/wrist_image_left": np.array(obs["image_wrist"], dtype=np.uint8),
        "observation/exterior_image_1_left": img_left,
        "observation/wrist_image_left": img_wrist,
        "observation/joint_position": np.array(obs["robot_state"], dtype=np.float32),
        "observation/gripper_position": np.array([obs["gripper_state"]], dtype=np.float32),  
        "prompt": prompt,
    }

    result = policy.infer(observation)
    action_list = result["actions"].tolist()
    end2 = time.time()
    print(f"Inference took {end2 - end1:.2f} seconds")
    print(f"Action at step {step}:", action_list[0])
    end = time.time()
    print(f"Step {step} took {end - start:.2f} seconds")


In [ ]:
import imageio
import cv2


gif_path = "pi0_fast_deploy_1.gif"
imageio.mimsave(gif_path, video_buffer, duration=0.5)
print(f"GIF saved:{gif_path}")

In [ ]:
from IPython.display import Image
Image(filename="pi0_fast_deploy_1.gif")